<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.2
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerous than some, and does not have waiting period for approval of new developers.

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access
- fill in the form fields at the bottom
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"

- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [1]:
#name 
# name (the name you gave to your app):- API for Python
# redirect URI:- http://127.0.0.1:1410
# personal use script (this is your OAuth 2 Client ID):- prIeDVIjWn-qRpmMRiZHXQ 
# secret (this is your OAuth 2 Secret):- uUnYvPxQBsnjVRY_J2bw1UYfO804Yg


In [2]:
!pip install praw

In [3]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [4]:
my_user_agent = 'windows-python-API for Python:v1.0 (by /u/API for Python)'   # your user Agent string goes in here
my_client_id = 'prIeDVIjWn-qRpmMRiZHXQ'   # your Client ID string goes in here
my_client_secret = 'uUnYvPxQBsnjVRY_J2bw1UYfO804Yg '   # your Secret string goes in here
redirect_uri = 'http://localhost:8080'

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [5]:
pwd()  # make sure your working directory is where the file is   

'C:\\Users\\Vinitha\\Desktop\\IOD\\Module_3\\Labs_3'

In [6]:
path_auth = "C:/Users/Vinitha/Desktop/IOD/Module_3/Labs_3/auth_reddit.json"
auth = json.loads(open (path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:

pp.pprint(auth)


{   'my_client_id': 'prIeDVIjWn-qRpmMRiZHXQ',
    'my_client_secret': 'uUnYvPxQBsnjVRY_J2bw1UYfO804Yg',
    'my_user_agent': 'windows-python-API for Python:v1.0 (by /u/API for '
                     'Python)'}


In [7]:
my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']
redirect_uri = 'http://localhost:8080' 

Security considerations:
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [8]:
reddit = praw.Reddit(client_id = "prIeDVIjWn-qRpmMRiZHXQ",
                     client_secret = "uUnYvPxQBsnjVRY_J2bw1UYfO804Yg",
                     redirect_uri= "http://127.0.0.1:1410",
                     user_agent = 'windows-python-API for Python:v1.0 (by /u/API for' 
                     'Python)')

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [9]:
reddit.  # Place cursor here and press Tab

SyntaxError: invalid syntax (1735263489.py, line 1)

In [10]:
subreddit_name = 'malaysia'
subreddit = reddit.subreddit(subreddit_name)

In [79]:
path_auth = "C:/Users/Vinitha/Desktop/IOD/Module_3/Labs_3/auth_reddit.json"
        
with open(path_auth) as f:
    auth_data = json.load(f)

# Initialize Reddit instance
reddit = praw.Reddit(
        client_id=auth_data['my_client_id'],
        client_secret=auth_data['my_client_secret'],
        redirect_uri=redirect_uri,
        user_agent=auth_data['my_user_agent']
    )

In [11]:
authorization_url = reddit.auth.url(scopes=['identity', 'read'], state='UniqueState', duration='temporary')

In [13]:
auth_url = reddit.auth.url(['identity'], 'unique_state_string', 'permanent')

print("Authorization URL:", auth_url)

Authorization URL: https://www.reddit.com/api/v1/authorize?client_id=prIeDVIjWn-qRpmMRiZHXQ&duration=permanent&redirect_uri=http%3A%2F%2F127.0.0.1%3A1410&response_type=code&scope=identity&state=unique_state_string


C:\Users\Vinitha\AppData\Local\Temp\ipykernel_13132\1958435228.py:1: DeprecationWarning: Positional arguments for 'Auth.url' will no longer be supported in PRAW 8.
Call this function with 'scopes', 'state', and 'duration' as keyword arguments.
  auth_url = reddit.auth.url(['identity'], 'unique_state_string', 'permanent')


In [81]:
print(f'Please authorize the app by visiting this URL:\n{authorization_url}')

Please authorize the app by visiting this URL:
https://www.reddit.com/api/v1/authorize?client_id=prIeDVIjWn-qRpmMRiZHXQ&duration=temporary&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity+read&state=UniqueState


In [14]:
reddit.auth.authorize()

TypeError: Auth.authorize() missing 1 required positional argument: 'code'

In [18]:
print(reddit.user.me())

None


In [19]:
comments = []

for comment in subreddit.comments(limit=1000):
        comments.append(comment)

In [20]:
reddit.subreddit(subreddit_name)

Subreddit(display_name='malaysia')

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic.

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [21]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
How do I stop numbers going below 0 in Python?
Does being an engineer/coder mean just looking up all the answers to your problems?
What is Python used for in Physics?
What are some common problems that can be solved mathematically, rather than programmatically?
How to make a web( I have my backend)
What's the best way to subsidize normal studying with studying with just your phone?
How to handle Python packages that are hardware and OS dependent?
How to resize Tab after move?
how to learn python beyond basics


Now retrieve 10 authors:

In [22]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
Ok_Start_6063
AccordingBeyond2985
creditcardmuncher
Hashi856
MaleficentAd1696
penis-learning
lynob
wallagix
Ok-Bullfrog-1113


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [23]:
submissions=reddit.subreddit('learnpython')

Why doesn't the next cell produce output?

In [25]:
for submission in submissions:
    submission.comments.replace_more(limit=None)  
    print("\nSubmission Title:", submission.title)
    print("Submission URL:", submission.url)

TypeError: 'Subreddit' object is not iterable

Print two comments associated with each of these submissions:

In [26]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

Dear python enthusiasts,

I am looking for an IT-"type" tool for automating python scripts. I have the following requirements

* It should be able to start/stop and restart python scripts
* Have levels where one script is started after another
* Show which scripts are alive
* Separated into frontend and backend, where Backend MUST be written in python doing the above three points
* MIT or BSD licensed (also pip installable is great!), so that I can add my own features

Frontend should do the following:

* See in a dashboard all controlled scripts (by this tool) and whether a script is running or not (I can also make one only if the backend is available)
* See also - in what all computers this tool has been installed

I could make my own, but that is not the point of this inquiry. I would like to save my work if any great tools already exists, especially the python backend, where people have thought about it even better.

Any suggestions welcome, thank you upfront.
I have been learning 

Referring to the API documentation, explore the submissions object and print some interesting data:

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request:

In [27]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

False


You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

In [28]:
pwd() 

'C:\\Users\\Vinitha\\Desktop\\IOD\\Module_3\\Labs_3'

In [29]:
path_auth = "C:/Users/Vinitha/Desktop/IOD/Module_3/Labs_3/auth_reddit.json"
auth = json.loads(open (path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:

pp.pprint(auth)


{   'my_client_id': 'prIeDVIjWn-qRpmMRiZHXQ',
    'my_client_secret': 'uUnYvPxQBsnjVRY_J2bw1UYfO804Yg',
    'my_user_agent': 'windows-python-API for Python:v1.0 (by /u/API for '
                     'Python)'}


>
>


>
>




---



---



> > > > > > > > > © 2024 Institute of Data


---



---



